In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [1]:
import pandas as pd

# Read CSV into Pandas dataframe

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select features

In [3]:
# Set features. This will also be used as your x values. Selected fields that are not labeled as  flag or variable errors (to avoid
# possible correlated variables)
selected_features = df[["koi_disposition","koi_period", "koi_time0bk","koi_impact", "koi_duration","koi_depth",
                                      "koi_prad", "koi_teq", "koi_insol", "koi_model_snr", "koi_steff","koi_slogg",
                                      "koi_srad", "ra", "dec", "koi_kepmag"]]
selected_features.head()

,koi_disposition,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,CONFIRMED,54.418383,162.513840,0.586,4.50700,874.8,2.83,443,9.11,25.8,5455,4.467,0.927,291.93423,48.141651,15.347
1,FALSE POSITIVE,19.899140,175.850252,0.969,1.78220,10829.0,14.60,638,39.30,76.3,5853,4.544,0.868,297.00482,48.134129,15.436
2,FALSE POSITIVE,1.736952,170.307565,1.276,2.40641,8079.2,33.46,1395,891.96,505.6,5805,4.564,0.791,285.53461,48.285210,15.597
3,CONFIRMED,2.525592,171.595550,0.701,1.65450,603.3,2.75,1406,926.16,40.9,6031,4.438,1.046,288.75488,48.226200,15.509
4,CONFIRMED,4.134435,172.979370,0.762,3.14020,686.0,2.77,1160,427.65,40.2,6046,4.486,0.972,296.28613,48.224670,15.714


# Neural Network

In [4]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

# Assign x and y values
X = selected_features.drop("koi_disposition", axis=1)
y = selected_features["koi_disposition"]

# Assign numeric values to string y values

from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

In [5]:
encoded_y

array([1, 2, 2, ..., 0, 2, 2])

In [6]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, encoded_y, random_state=1)

In [7]:
X_train.head()

,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
3563,10.548413,139.064020,1.0170,1.8720,102.9,3.89,899,154.84,11.7,6047,4.387,1.092,298.09543,44.737061,13.204
4099,24.754385,140.207320,0.7090,3.3900,593.3,2.10,491,13.70,18.0,4852,4.519,0.804,295.73535,42.576248,15.514
5460,1.057336,131.792007,0.2620,1.5795,47337.0,14.59,1276,623.51,476.0,4664,4.594,0.683,292.18417,49.310040,15.414
1091,201.118319,187.569860,0.0010,10.3280,584.8,2.28,300,1.92,34.7,5646,4.447,0.954,283.11377,48.131390,13.328
5999,91.649983,175.715600,0.2136,10.2940,193.6,2.27,568,24.57,8.7,6705,4.145,1.608,294.93198,39.812420,12.964


In [8]:
# Create a MinMaxScaler model and fit it to the training data

X_scaler = MinMaxScaler().fit(X_train)

In [9]:
# Transform the training and testing data using the X_scaler

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# first, create a normal neural network with 15 inputs, 6 hidden nodes, and 3 outputs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=6, activation='relu', input_dim=15))
model.add(Dense(units=3, activation='softmax'))

In [11]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
# One-hot encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [13]:
# Fit the model to the training data
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Train on 5243 samples
Epoch 1/100
5243/5243 - 0s - loss: 1.0637 - accuracy: 0.4797
Epoch 2/100
5243/5243 - 0s - loss: 1.0342 - accuracy: 0.4936
Epoch 3/100
5243/5243 - 0s - loss: 1.0193 - accuracy: 0.4948
Epoch 4/100
5243/5243 - 0s - loss: 1.0049 - accuracy: 0.4963
Epoch 5/100
5243/5243 - 0s - loss: 0.9885 - accuracy: 0.4999
Epoch 6/100
5243/5243 - 0s - loss: 0.9702 - accuracy: 0.5085
Epoch 7/100
5243/5243 - 0s - loss: 0.9517 - accuracy: 0.5218
Epoch 8/100
5243/5243 - 0s - loss: 0.9351 - accuracy: 0.5363
Epoch 9/100
5243/5243 - 0s - loss: 0.9213 - accuracy: 0.5503
Epoch 10/100
5243/5243 - 0s - loss: 0.9103 - accuracy: 0.5619
Epoch 11/100
5243/5243 - 0s - loss: 0.9017 - accuracy: 0.5672
Epoch 12/100
5243/5243 - 0s - loss: 0.8935 - accuracy: 0.5712
Epoch 13/100
5243/5243 - 0s - loss: 0.8866 - accuracy: 0.5772
Epoch 14/100
5243/5243 - 0s - loss: 0.8811 - accuracy: 0.5813
Epoch 15/100
5243/5243 - 0s - loss: 0.8755 - accuracy: 0.5850
Epoch 16/100
5243/5243 - 0s - loss: 0.8708 - accuracy: 0.

# Deep Learning model

In [14]:
# Setting model parameters

deep_model = Sequential()
deep_model.add(Dense(units=15, activation='relu', input_dim=15))
deep_model.add(Dense(units=15, activation='relu'))
deep_model.add(Dense(units=20, activation='relu'))
deep_model.add(Dense(units=20, activation='relu'))
deep_model.add(Dense(units=20, activation='relu'))
deep_model.add(Dense(units=3, activation='softmax'))

In [15]:
# Compiling deep model

deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Train on 5243 samples
Epoch 1/100
5243/5243 - 0s - loss: 1.0463 - accuracy: 0.4835
Epoch 2/100
5243/5243 - 0s - loss: 0.9781 - accuracy: 0.5121
Epoch 3/100
5243/5243 - 0s - loss: 0.9039 - accuracy: 0.5648
Epoch 4/100
5243/5243 - 0s - loss: 0.8683 - accuracy: 0.5842
Epoch 5/100
5243/5243 - 0s - loss: 0.8446 - accuracy: 0.6040
Epoch 6/100
5243/5243 - 0s - loss: 0.8322 - accuracy: 0.6107
Epoch 7/100
5243/5243 - 0s - loss: 0.8254 - accuracy: 0.6065
Epoch 8/100
5243/5243 - 0s - loss: 0.8213 - accuracy: 0.6140
Epoch 9/100
5243/5243 - 0s - loss: 0.8196 - accuracy: 0.6142
Epoch 10/100
5243/5243 - 0s - loss: 0.8039 - accuracy: 0.6245
Epoch 11/100
5243/5243 - 0s - loss: 0.7983 - accuracy: 0.6275
Epoch 12/100
5243/5243 - 0s - loss: 0.7924 - accuracy: 0.6330
Epoch 13/100
5243/5243 - 0s - loss: 0.7873 - accuracy: 0.6338
Epoch 14/100
5243/5243 - 0s - loss: 0.7791 - accuracy: 0.6374
Epoch 15/100
5243/5243 - 0s - loss: 0.7820 - accuracy: 0.6298
Epoch 16/100
5243/5243 - 0s - loss: 0.7773 - accuracy: 0.

# Comparing both neural and deep learning models

In [16]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1748/1748 - 0s - loss: 0.7623 - accuracy: 0.6424
Normal Neural Network - Loss: 0.7623072055166319, Accuracy: 0.6424484848976135


In [17]:
model_loss, model_accuracy = deep_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1748/1748 - 0s - loss: 0.7086 - accuracy: 0.6745
Deep Neural Network - Loss: 0.7086345197407005, Accuracy: 0.6744851469993591


# Saving the model

In [18]:
# Save the model
model.save("neuro_exoplanets_trained.h5")

In [19]:
deep_model.save("deep_exoplanets_trained.h5")